In [110]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('fivethirtyeight')

from os import listdir
from os.path import isfile, join

import re

In [111]:
def removeExtension(file_name):
    idx = -1
    for i, c in enumerate(file_name):
        if c == '.':
            idx = i
            break
    if idx >= 0:
        return file_name[:idx]
    return file_name

In [112]:
SUBMISSIONS_DIR = './submissions/'
MATCH_RESULTS_DIR = './match_results/'

In [113]:
files = sorted([f for f in listdir(SUBMISSIONS_DIR) if isfile(join(SUBMISSIONS_DIR, f)) and f[-1] != 'c'])
names = sorted([removeExtension(f) for f in files])

teams = list(zip(files, names))

In [134]:
match_results_df = pd.DataFrame(columns=['Team', 'Opponent', 'Wins', 'Losses', 'Ties', 'Invalid'])

In [135]:
def process_raw_outcome(outcome):
    return outcome.replace('\n', '')

for i in range(len(teams)):
    for j in range(i + 1, len(teams)):
        if i == j:
            continue
        
        file_a, name_a = teams[i]
        file_b, name_b = teams[j]
        
        # Get match results for team a vs team b
        match_data_file_name = name_a + '_vs_' + name_b + '.dat'
        try:
            match_data_file = open(join(MATCH_RESULTS_DIR, match_data_file_name), 'r')
        except FileNotFoundError:
            continue
        outcomes = [process_raw_outcome(l) for l in match_data_file]
        match_data_file.close()
        
        # Get DataFrame props
        num_rows, num_cols = match_results_df.shape
        
        # Insert row if doesn't exist or reset row
        index_a, index_b = -1, -1
        for k in range(num_rows):
            row = match_results_df.loc[k]
            if row.Team == name_a and row.Opponent == name_b:
                index_a = k
            if row.Team == name_b and row.Opponent == name_a:
                index_b = k
        if index_a == -1 and index_b == -1:
            index_a, index_b = num_rows, num_rows + 1
        elif index_a == -1:
            index_a = num_rows
        elif index_b == -1:
            index_b == num_rows
        
        win, loss, tied, invalid = 0, 0, 0, 0
        for outcome in outcomes:
            if outcome == name_a:
                win += 1
            elif outcome == name_b:
                loss += 1
            elif outcome == 'tied':
                tied += 1
            elif outcome == 'invalid':
                invalid += 1
            else:
                raise Error('Invalid match outcome')
        match_results_df.loc[index_a] = [name_a, name_b, win, loss, tied, invalid]
        match_results_df.loc[index_b] = [name_b, name_a, loss, win, tied, invalid]

In [139]:
match_results_df.shape

(1756, 6)

In [136]:
match_results_df.head()

,Team,Opponent,Wins,Losses,Ties,Invalid
0,PacBot_Kevin_Cao,baselineTeam2_Rachel_Li,9,3,0,3
1,baselineTeam2_Rachel_Li,PacBot_Kevin_Cao,3,9,0,3
2,PacBot_Kevin_Cao,baselineTeam2_Raghav_Anand,7,8,0,0
3,baselineTeam2_Raghav_Anand,PacBot_Kevin_Cao,8,7,0,0
4,PacBot_Kevin_Cao,cleverTeam_Dylan_Rowe,10,5,0,0


In [137]:
match_results_df[match_results_df['Invalid'] > 10]

,Team,Opponent,Wins,Losses,Ties,Invalid
6,PacBot_Kevin_Cao,contest_ANISH_SAHA,0,0,0,15
7,contest_ANISH_SAHA,PacBot_Kevin_Cao,0,0,0,15
8,PacBot_Kevin_Cao,contest_ANISH_SAHA_1,0,0,0,15
9,contest_ANISH_SAHA_1,PacBot_Kevin_Cao,0,0,0,15
10,PacBot_Kevin_Cao,coolTeam_SONTHAYA_VISUTHIKRAISEE,1,0,0,14
11,coolTeam_SONTHAYA_VISUTHIKRAISEE,PacBot_Kevin_Cao,0,1,0,14
20,PacBot_Kevin_Cao,myTeam2_SEAN_ADAM_MEDIN,0,0,0,15
21,myTeam2_SEAN_ADAM_MEDIN,PacBot_Kevin_Cao,0,0,0,15
34,PacBot_Kevin_Cao,myTeam3_ALI_KHALEEL_AHMED_4,0,0,2,13
35,myTeam3_ALI_KHALEEL_AHMED_4,PacBot_Kevin_Cao,0,0,2,13


In [140]:
match_results_df[match_results_df['Team'] == 'myTeam_Laura_Smith']

,Team,Opponent,Wins,Losses,Ties,Invalid
77,myTeam_Laura_Smith,PacBot_Kevin_Cao,1,0,0,14
165,myTeam_Laura_Smith,baselineTeam2_Rachel_Li,9,4,2,0
251,myTeam_Laura_Smith,baselineTeam2_Raghav_Anand,9,6,0,0
335,myTeam_Laura_Smith,cleverTeam_Dylan_Rowe,9,5,1,0
417,myTeam_Laura_Smith,contest_ANISH_SAHA,0,0,0,15
497,myTeam_Laura_Smith,contest_ANISH_SAHA_1,0,0,0,15
575,myTeam_Laura_Smith,coolTeam_SONTHAYA_VISUTHIKRAISEE,6,4,5,0
651,myTeam_Laura_Smith,myTeam2_4foodNum_ALI_KHALEEL_AHMED,8,4,1,2
725,myTeam_Laura_Smith,myTeam2_5foodNum_ALI_KHALEEL_AHMED,6,7,2,0
797,myTeam_Laura_Smith,myTeam2_ALI_KHALEEL_AHMED,9,5,0,1
